<a href="https://colab.research.google.com/github/kasodeep/inheritance-project/blob/main/llama_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tiktoken

In [2]:
!pip install langchain
!pip install fpdf
!pip install ctransformers
!pip install pypdf

   ---------------------------------------- 0.0/283.9 kB ? eta -:--:--
   ------------------------------- -------- 225.3/283.9 kB 6.9 MB/s eta 0:00:01
   ---------------------------------------- 283.9/283.9 kB 5.8 MB/s eta 0:00:00


In [27]:
#imports
import langchain
from torch import cuda, bfloat16
from fpdf import FPDF
import torch
import transformers
from transformers import AutoTokenizer
from time import time
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader,PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain,ConversationalRetrievalChain,StuffDocumentsChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import CTransformers
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain import PromptTemplate, LLMChain

In [28]:
#loading downloaded llm suitable for local usage,temperature(entropy/randomness in answer):1e-2
# But first you would need to mount your Google Drive to Google Colab.
llm = CTransformers(model=r"/content/drive/MyDrive/llama-2-13b.ggmlv3.q2_K.bin", model_type="llama", streaming=True,
                    callbacks=[StreamingStdOutCallbackHandler()],
                    config={'max_new_tokens':4096,'temperature':0.01, 'context_length':4096})

In [29]:
def load_documents(file_path):
    # Assuming you have a function to load documents (e.g., PyPDFLoader)
    loader = PyPDFLoader(file_path)
    docs = loader.load()
    return docs

def summary_generation(file_path):
    # Load documents
    docs = load_documents(file_path)

    # Map
    map_template = """The following is a set of documents
    {docs}
    Based on this list of docs, please identify the main themes and concepts
    Expand the description of each topic and concept for 2-3 lines that should include its basic descriptions, key points, and formulas if any.
    Helpful Answer:"""
    map_prompt = PromptTemplate.from_template(map_template)
    map_chain = LLMChain(llm=llm, prompt=map_prompt)

    # Reduce
    reduce_template = """The following is a set of summaries:
    {docs}
    Take these and distill it into a final, consolidated summary of the main topics and concepts that should include definitions and formulas of the concepts. Mention all the key points and formulas related to a concept.
    Expand the description of each topic and concept for 2-3 lines.
    Helpful Answer:"""
    reduce_prompt = PromptTemplate.from_template(reduce_template)

    # Run chain
    reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

    # Takes a list of documents, combines them into a single string, and passes this to an LLMChain
    combine_documents_chain = StuffDocumentsChain(
        llm_chain=reduce_chain, document_variable_name="docs"
    )

    # Combines and iteratively reduces the mapped documents
    reduce_documents_chain = ReduceDocumentsChain(
        # This is the final chain that is called.
        combine_documents_chain=combine_documents_chain,
        # If documents exceed context for `StuffDocumentsChain`
        collapse_documents_chain=combine_documents_chain,
        # The maximum number of tokens to group documents into.
        token_max=4000,
    )

    # Combining documents by mapping a chain over them, then combining results
    map_reduce_chain = MapReduceDocumentsChain(
        # Map chain
        llm_chain=map_chain,
        # Reduce chain
        reduce_documents_chain=reduce_documents_chain,
        # The variable name in the llm_chain to put the documents in
        document_variable_name="docs",
        # Return the results of the map steps in the output
        return_intermediate_steps=False,
    )

    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500, chunk_overlap=50
    )
    split_docs = text_splitter.split_documents(docs)
    all_summaries = map_reduce_chain.run(split_docs)
    print(all_summaries)

    return all_summaries



In [ ]:
summary_generation(r"/content/transcript_output.pdf")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



        The following is a set of documents
            Language Code: en
Pakistani journalist aru kazmi is also now joining us thank you so much for your time uh by and large
the response by far of Pakistan is being seen as a Meek sort of a response statement being issued
that two children have been killed three women three girls have been injured also how do you
look into this response of Pakistan um I think Pakistan is not informing the right news uh as a
journalist I feel that because how this possible that Iran attack Pakistan to to the militant group and it
it kills two children only and three injured so there's something we are waiting for the full news and I
think the complete uh news we will get after few hours because it's too early that I comment on this
and uh but still foreign office issued this uh statement or uh s this is what I want to say that yes this
militant group was involved and attack Iran several times so that's why we are not surprised that Iran
tried to kill 

-------------------------------------------------------------------------

In [ ]:
!pip install fpdf
!pip install pytube
!pip install youtube-transcript-api
!pip install fpdf
!pip install youtube_transcript_api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 838.2 kB/s eta 0:00:00


In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi
from fpdf import *

def get_transcript(youtube_url, output_pdf_path):
    video_id = youtube_url.split("v=")[-1]
    transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)

    # Try fetching the manual transcript
    try:
        transcript = transcript_list.find_manually_created_transcript()
        language_code = transcript.language_code  # Save the detected language
    except:
        # If no manual transcript is found, try fetching an auto-generated transcript in a supported language
        try:
            generated_transcripts = [trans for trans in transcript_list if trans.is_generated]
            transcript = generated_transcripts[0]
            language_code = transcript.language_code  # Save the detected language
        except:
            # If no auto-generated transcript is found, raise an exception
            raise Exception("No suitable transcript found.")

    full_transcript = " ".join([part['text'] for part in transcript.fetch()])

    # Save the transcript to a PDF file
    save_to_pdf(full_transcript, language_code, output_pdf_path)

    return full_transcript, language_code  # Return both the transcript and detected language

def save_to_pdf(transcript, language_code, output_pdf_path):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    pdf.cell(200, 10, txt=f"Language Code: {language_code}", ln=True, align='C')
    pdf.ln(10)

    pdf.multi_cell(0, 10, txt=transcript)

    pdf.output(output_pdf_path)

# Example usage
output_pdf_path = "transcript_output.pdf"
get_transcript("https://www.youtube.com/watch?v=tsJdXpnl48g&list=RDNStsJdXpnl48g&start_radio=1", output_pdf_path)

("Pakistani journalist aru kazmi is also now joining us thank you so much for your time uh by and large the response by far of Pakistan is being seen as a Meek sort of a response statement being issued that two children have been killed three women three three girls have been injured also how do you look into this response of Pakistan um I think Pakistan is not informing the right news uh as a journalist I feel that because how this possible that Iran attack Pakistan to to the militant group and it it kills two children only and three injured so there's something we are waiting for the full news and I think the complete uh news we will get after few hours because it's too early that I comment on this and uh but still foreign office issued this uh statement or uh s this is what I want to say that yes this militant group was involved and attack Iran several times so that's why we are not surprised that Iran tried to kill them or try to hit them but yes uh we are waiting for ispr and also